# Threat Modeling - Overview

Threat modeling is a structured way to identify, prioritize, and mitigate security risks before they become incidents. It turns "what could go wrong?" into concrete threats, assumptions, and controls.

## Purpose
- Reduce security surprises by surfacing risks early.
- Align engineering, security, and product on what matters most.
- Produce actionable mitigations tied to system components.

## Key questions this section answers
- What are the critical assets and trust boundaries?
- Which threats are most likely and most damaging?
- What controls reduce risk to an acceptable level?

## Core frameworks
- STRIDE (spoofing, tampering, repudiation, information disclosure, denial of service, elevation of privilege)
- LINDDUN for privacy threats
- PASTA and Trike for risk-driven analysis
- Attack trees and MITRE ATT&CK for adversary behaviors

## Outputs you should expect
- A data flow diagram (DFD) with trust boundaries
- A threat register with likelihood, impact, and mitigations
- Clear ownership and follow-up actions


## Process at a glance
1. Define scope and assets.
2. Map data flows and trust boundaries.
3. Enumerate threats by component and data flow.
4. Rate risk and prioritize.
5. Choose mitigations and validate with tests.
6. Review changes as the system evolves.


## Risk scoring quick model
A simple expected-loss proxy:

$$Risk = Likelihood \times Impact$$

Use a 1-5 scale for each and focus on high-risk items first. The goal is consistency, not false precision.


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"

threats = pd.DataFrame(
    [
        {
            "component": "Web app",
            "threat": "Credential stuffing",
            "category": "Spoofing",
            "likelihood": 4,
            "impact": 4,
            "mitigation": "MFA, rate limits, bot detection",
        },
        {
            "component": "API",
            "threat": "Parameter tampering",
            "category": "Tampering",
            "likelihood": 3,
            "impact": 4,
            "mitigation": "Schema validation, authZ checks",
        },
        {
            "component": "Database",
            "threat": "Privilege escalation",
            "category": "Elevation of privilege",
            "likelihood": 2,
            "impact": 5,
            "mitigation": "Least privilege, row-level policies",
        },
        {
            "component": "Auth service",
            "threat": "Token replay",
            "category": "Repudiation",
            "likelihood": 3,
            "impact": 3,
            "mitigation": "Short TTL, nonce, audit logs",
        },
        {
            "component": "Edge",
            "threat": "Volumetric DDoS",
            "category": "Denial of service",
            "likelihood": 3,
            "impact": 5,
            "mitigation": "WAF, autoscale, rate limits",
        },
    ]
)

threats["risk"] = threats["likelihood"] * threats["impact"]


def bucket(score: int) -> str:
    if score >= 16:
        return "critical"
    if score >= 12:
        return "high"
    if score >= 8:
        return "medium"
    return "low"


threats["level"] = threats["risk"].map(bucket)
threats.sort_values("risk", ascending=False)


In [ ]:
import numpy as np

likelihood = np.arange(1, 6)
impact = np.arange(1, 6)
score = np.outer(likelihood, impact)

fig = go.Figure(
    data=go.Heatmap(
        z=score,
        x=impact,
        y=likelihood,
        colorscale="YlOrRd",
        colorbar=dict(title="Risk"),
    )
)
fig.update_layout(
    title="Risk matrix (likelihood x impact)",
    xaxis_title="Impact",
    yaxis_title="Likelihood",
    yaxis_autorange="reversed",
)
fig.show()


## STRIDE mapping example
Map each component to the STRIDE categories to force breadth of thinking. The goal is to list concrete threats and then map them to mitigations.


In [ ]:
stride = pd.DataFrame(
    [
        {
            "component": "Web app",
            "stride": "Spoofing",
            "example": "Session hijacking",
            "mitigation": "Secure cookies, CSRF protection",
        },
        {
            "component": "Web app",
            "stride": "Information disclosure",
            "example": "Leaky error pages",
            "mitigation": "Generic error responses, logging policy",
        },
        {
            "component": "API",
            "stride": "Tampering",
            "example": "Insecure direct object access",
            "mitigation": "AuthZ checks per resource",
        },
        {
            "component": "API",
            "stride": "Repudiation",
            "example": "No audit trail",
            "mitigation": "Immutable audit logs",
        },
        {
            "component": "Database",
            "stride": "Elevation of privilege",
            "example": "Wide service account rights",
            "mitigation": "Least privilege roles",
        },
    ]
)

stride


## Data flow diagram (simplified)
Use a data flow diagram to identify trust boundaries, external dependencies, and risky data paths. The shapes below are a lightweight example you can adapt to your system.


In [ ]:
nodes = {
    "User": (0, 0),
    "Web app": (1, 0),
    "API": (2, 0),
    "Auth": (2, 1),
    "Database": (3, 0),
}

edges = [
    ("User", "Web app"),
    ("Web app", "API"),
    ("API", "Database"),
    ("API", "Auth"),
]

fig = go.Figure()

for start, end in edges:
    x0, y0 = nodes[start]
    x1, y1 = nodes[end]
    fig.add_trace(
        go.Scatter(
            x=[x0, x1],
            y=[y0, y1],
            mode="lines",
            line=dict(color="#6b7280", width=2),
            hoverinfo="none",
            showlegend=False,
        )
    )

fig.add_trace(
    go.Scatter(
        x=[pos[0] for pos in nodes.values()],
        y=[pos[1] for pos in nodes.values()],
        text=list(nodes.keys()),
        mode="markers+text",
        textposition="bottom center",
        marker=dict(size=18, color="#2563eb"),
        showlegend=False,
    )
)

fig.add_shape(
    type="rect",
    x0=0.5,
    y0=-0.5,
    x1=3.5,
    y1=1.5,
    line=dict(color="#ef4444", dash="dash"),
)

fig.update_layout(
    title="Example data flow with a trust boundary",
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    margin=dict(l=20, r=20, t=50, b=20),
)
fig.show()


## Mitigations and controls
- Preventive: authN/authZ, least privilege, input validation, secure defaults.
- Detective: logging, alerting, anomaly detection, audit trails.
- Resilient: rate limits, circuit breakers, autoscaling, graceful degradation.
- Recovery: backups, incident runbooks, tabletop exercises.


## Practical tips
- Keep models small and current; update on architecture changes.
- Pair security and engineering in the same session to avoid blind spots.
- Tie every mitigation to an owner and a concrete test.

## Exercises
1. Draw a DFD for a login flow and identify two trust boundaries.
2. Produce a STRIDE table for a file upload service.
3. Convert your top three risks into specific backlog items.

## Further reading
- STRIDE, LINDDUN, PASTA, Trike, and MITRE ATT&CK references
- NIST risk management guidance (SP 800-30)
- OWASP ASVS for control-level checklists
